## VISUALIZATION PROJECT Geospatial Business Intelligence (BI)

El objetivo del proyecto es establecer una localización (o posibles localizaciones) para una nueva empresa de software orientada a la creación de videojuegos. Para determinar dicha localización se proponen ciertas condiciones:
- Los roles dentro de la nueva compañía son: 20 desarrolladores, 20 diseñadores/creativos/UX-UI y 10 ejecutivos/managers.
- Deben existir ingenieros de software trabajando en los alrededores de la localización.
- Debe existir un buen ratio entre grandes empresas y start-ups.
- Deben existir empresas en los alrededores que cubran los intereses del equipo.
- Se debe evitar compañias antiguas.

Para abordar el problema se ha optado por definir 5 criterios que cubren las exigencias que debe tener la localización óptima de la nueva empresa de video juegos, los cuales incluyen los criterios planteados en el planteamiento del problema. Los criterios son los siguientes:
- Criterio 1 (Capability): se refiere a la disponibilidad de profesionales cualificados.
- Criterio 2 (Cost): se refiere a la disponibilidad de suelo barato.
- Criterio 3 (Communications): se refiere a la existencia de un cluster de empresas relevantes en los alrededores.
- Criterio 4 (Capacity): se refiere a la disponibilidad de nuevos profesionales con formación relevante.
- Criterio 5 (Culture): se refiere a la existencia de un cluster de empresas que aporten una cultura empresarial atractiva.

Para desarrollar este enfoque se ha creado un pipeline para cada criterio de los que se extraerá un dataset (json) que se importará en Tableau. La información que se ilustrará en Tableau mostrará diferentes localizaciones (latitud/longitud) que cumplen con cada criterio, lo cual permitirá valorar más opciones para la toma de decisión.

Documentos entregados:
- main.ipynb (documento principal donde se ha desarrollado el código inicial. Este es un documento de trabajo que luego se usará de base para crear el dataset para cada criterio).
- criterio1.ipynb (código para generar las localizaciones más óptimas según lo establecido en el criterio 'Capability') => capability.json, df_capability.json
- criterio2.ipynb (código para generar las localizaciones más óptimas según lo establecido en el criterio 'Cost') => cost.json, df_cost.json
- criterio3.ipynb (código para generar las localizaciones más óptimas según lo establecido en el criterio 'Communications') => communications.json, df_communications.json
- criterio4.ipynb (código para generar las localizaciones más óptimas según lo establecido en el criterio 'Capacity') => capacity.json, df_capacity.json
- criterio5.ipynb (código para generar las localizaciones más óptimas según lo establecido en el criterio 'Culture') => culture.json, df_culture.json
- tablea_project.json (fichero definitivo para importar en Tableu).
- Visualización en Tableau Public: https://public.tableau.com/profile/potacho

In [1]:
#Importo librerias.
import pymongo
import pandas as pd
import re

#Creo cursor y dataframe inicial.
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies
cursor = db.companies.find()
data = list(cursor)
df = pd.DataFrame(data)

In [30]:
#Verifico que se han importado todos los documentos.
cursor = db.companies.find()
data = list(cursor)
#print(len(data))

In [31]:
#Funciones para analizar datos.
def q_qty_1(atr1,var1):
    q = db.companies.find({atr1:var1})
    return len(list(map(lambda r:r,list(q))))
atr1 = 'category_code'
var1 = 'web'
#print(q_qty_1(atr1,var1))

In [32]:
#Funciones para analizar datos.
def q_1(atr1,var1):
    q = db.companies.find({atr1:var1})
    return list(map(lambda r:[(r['name']),(r['offices'])],list(q)))
atr1 = 'category_code'
var1 = 'web'
#display(q_1(atr1,var1))

In [33]:
#Creo dataframe con los datos
df = pd.DataFrame(data)
#display(df.dtypes)
columns = df.dtypes
#print(columns)

In [34]:
#Extraigo columnas para tratamiento (para no copiarlas a mano)
#g = df.columns.to_series().groupby(df.dtypes).groups
#{k.name: v for k, v in g.items()}

In [35]:
#Clasificación de columnas originales para limpieza del dataframe
col_float = ['deadpooled_day','deadpooled_month','deadpooled_year','founded_day','founded_month',
             'founded_year','number_of_employees']
col_obj = ['_id','acquisition','acquisitions','alias_list','blog_feed_url','blog_url','category_code',
           'competitions','created_at','crunchbase_url','deadpooled_url','description','email_address',
        'external_links','funding_rounds','homepage_url','image','investments','ipo','milestones', 
           'name','offices','overview','partners','permalink','phone_number','products','providerships',
        'relationships','screenshots','tag_list','total_money_raised','twitter_username','updated_at',
           'video_embeds']
col_drop1 = ['deadpooled_day','deadpooled_month','founded_day','founded_month','alias_list','blog_feed_url',
             'blog_url','created_at','crunchbase_url','deadpooled_url','email_address','external_links',
            'image','permalink','phone_number','screenshots','tag_list','updated_at','video_embeds']
col_drop2 = ['acquisition','acquisitions','competitions','description','funding_rounds','homepage_url',
             'investments','ipo','milestones','partners','products','providerships','relationships',
             'twitter_username','overview',]
col_ok = ['deadpooled_year','founded_year','number_of_employees','_id','category_code','name','offices',
          'total_money_raised']

print(len(col_float)+len(col_obj),len(col_drop1)+len(col_drop2)+len(col_ok))

42 42


In [52]:
#Elimino columnas con valores que no me interesan
df_drop1 = df.drop(col_drop1, axis=1)
df_drop2 = df_drop1.drop(col_drop2, axis=1)
#Elijo solo las empresas con deadpooled_year == null
df_deadnull = df_drop2[df_drop2['deadpooled_year'].isnull()]
#Elimino filas con valores nulos de las columnas que me interesan
df_nulls = df_deadnull.dropna(subset=['founded_year','number_of_employees','name','offices',
          'total_money_raised'])
display(len(df_nulls))
#display(df_nulls.head())

7934

In [53]:
#Data frames por tipo de empresas y elimino las columnas _id y deadpooled_year 
col_cat = ['games_video','web','software']
df_cat = df_nulls[df_nulls['category_code'].isin(col_cat)]\
.drop(['_id','deadpooled_year'], axis=1)#.sort_values(by=['category_code','number_of_employees'])
print(len(df_cat))
#display(df_cat.head())

3752


In [38]:
#Defino funciones para la modificación de los datos de las columnas a analizar.

#Transforma un dato tipo float a uno tipo int.
def float_to_int(flt):
    integer = int(flt)
    return integer

#Transforma un dato tipo str en int devolviendo solo el número.
def str_to_int(strg):
    string = re.findall('\d+', strg )
    integer = int(string[0])*1000
    return integer


In [55]:
#Elimino registros irrelevantes (tomando solo los registros que me interesan)
df_regout = df_cat.copy()
df_regout['total_money_raised'] = df_regout['total_money_raised'].apply(str_to_int)
df_regout['founded_year'] = df_regout['founded_year'].apply(float_to_int)
#Elimino registros según criterios 
df_regout = df_regout[df_regout.total_money_raised != 0]
df_regout = df_regout[df_regout.number_of_employees != 0]
df_regout = df_regout[df_regout['offices'].map(len) > 0]
print(len(df_regout))
#display(df_regout.head())

768


In [40]:
#Verifico estructura del diccionario
#print(df_regout['offices']).iloc[100])

In [41]:
#d = {'type': 'Point', 'coordinates': [None, None]}
##s = d['coordinates']
#lon = d['coordinates'][0]
#if lon == None:
#    print('No hay valor de longitud para este registro')

In [42]:
#Transforma los datos de ubicación de las oficinas en formato GeoJson y selecciona solo la primera ubicación
def nested_to_list(dicts):
    lat_lon = [{"type":"Point","coordinates":[l['longitude'],l['latitude']]} for l in dicts]
    lst = lat_lon[0]
    long = lst['coordinates'][0]
    if long != None:
        return lst
    else:
        return 0

#print(nested_to_list([{'description': 'New York Office', 'address1': '', 'address2': '', 'zip_code': '', 'city': 'New York', 'state_code': None, 'country_code': 'USA', 'latitude': 40.756054, 'longitude': -73.986951},{'description': 'Barcelona Headquarters', 'address1': '', 'address2': '', 'zip_code': '', 'city': 'Barcelona', 'state_code': None, 'country_code': 'ESP', 'latitude': 41.387917, 'longitude': 2.1699187}]))

In [43]:
#Transformo valores de columnas aplicando las funciones anteriormente definidas.
df_colout = df_regout.copy()
df_colout['offices'] = df_colout['offices'].apply(nested_to_list)
print(len(df_colout))
#display(df_colout.head())

768


In [44]:
#print(df_colout['offices'])

In [45]:
#Elimino valores de GeoJson nulos
df_geoout = df_colout.copy()
df_geoout = df_geoout[df_geoout.offices != 0]
print(len(df_geoout))
display(df_geoout.head())

603


,category_code,founded_year,name,number_of_employees,offices,total_money_raised
6,web,2006,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,...",16000
20,web,2002,Plaxo,50.0,"{'type': 'Point', 'coordinates': [-122.055197,...",28000
23,web,1995,eBay,15000.0,"{'type': 'Point', 'coordinates': [-121.930035,...",6000
28,web,2007,Mahalo,40.0,"{'type': 'Point', 'coordinates': [-118.487267,...",21000
31,games_video,2006,Kyte,40.0,"{'type': 'Point', 'coordinates': [-122.409173,...",23000


In [46]:
#print(df_geoout['offices'])

In [47]:
df_geoout.to_json('prueba.json', orient="records", lines=True)

In [56]:
def concentration(company):
    cursor_near = db.fase1.find({
      "offices": {
        "$near": {
          "$geometry": {
            "type": "Point",
            "coordinates": [company['offices']['coordinates'][0], company['offices']['coordinates'][1]]
          },
          "$minDistance": 0,
          "$maxDistance":9000
        }
      }
    })
    return list(cursor_near)

In [57]:
#Hago loop sobre las empresas
cursor_geojson = db.fase1.find()
clusters = []
for element in cursor_geojson:
    clusters.append(len(concentration(element)))


display(len(clusters))

2305

In [58]:
print(max(clusters))
#print(clusters)


132


In [51]:
target = list(db.fase1.find())
targets = []
for i in range(len(clusters)):
    if clusters[i] == max(clusters):
        targets.append([target[i]['offices']['coordinates'][0], target[i]['offices']['coordinates'][1]])

print(targets)

[[-122.457454, 37.79851]]


# CONCLUSIONES

Una vez generados todos los ficheros .json para cada uno de los criterios, se genera un fichero final con todos los datos de localización.

In [12]:
#Creación de dataset definitivo.
df_capability = pd.read_json('df_capability.json', orient="records", lines=True)
df_cost = pd.read_json('df_cost.json', orient="records", lines=True)
df_communications = pd.read_json('df_communications.json', orient="records", lines=True)
df_capacity = pd.read_json('df_capacity.json', orient="records", lines=True)
df_culture = pd.read_json('df_culture.json', orient="records", lines=True)
df_tableau = df_capability.append(df_cost.append(df_communications.append(df_capacity.append(df_culture))))

#Visualización del dataframe final.
display(df_tableau)

,Criteria,Latitude,Longitude
0,Capability,40.726155,-73.995625
1,Capability,40.722655,-73.998730
2,Capability,40.723384,-74.001704
3,Capability,40.717248,-74.002662
4,Capability,40.739930,-73.993049
5,Capability,40.754762,-73.988543
6,Capability,40.738476,-73.998726
7,Capability,40.752672,-73.975930
8,Capability,40.743877,-73.986180
0,Cost,40.743148,-73.993429


In [13]:
#Creación de fichero .json para visualización en Plateau.
df_tableau.to_json('tableau_project.json', orient="records", lines=True)

La visualización generada corresponde a un radio de 5km. Se hicieron varias pruebas con varios radios y el lugar donde se concentran al menos 4 de los criterios cambia de NYC (5km) a San Francisco (10km). Como conclusión final comentar que se deben realizar más pruebas para establecer distancias óptimas para cada criterio y así tomar una decisión final. También sería necesario analizar en detalle los diferentes parámetros de cada criterio y refactorizar el código para mayor orden y mejor manipulación de diferentes parámentros bajo el mismo enfoque.







**OGM**

## NOTAS/ERRORES/INTENTOS

In [ ]:
#df_off = json_normalize(df_cat, 'offices')
#display(df_off.head())

In [ ]:
df_fase1 = pd.DataFrame(list(cursor))
print(len(df_fase1))
display(df_fase1.head())

In [ ]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[0],array[1]]
    }

In [ ]:
#Esto es para buscar con regex de mongo
#{ name: { $in: [ /^key./i ] } }    { tag_list: { $in: [ /.games./i ] } }  
#{ updated_at: { $in: [ /2008$/ ] } } 
#Para descartar arrays vacios en mongo
#{offices:{ $gt: [] } } (hay otras opciones en un link en pendientes)

In [ ]:
def convertRestaurant(restaurant):
    return {
        "name": restaurant["name"],
        "cuisine":restaurant["cuisine"],
        "position": toGeoJSON(restaurant["address"]["coord"])
    }


In [ ]:
#Elimino rows que no me aportan nada
#df_values = df_nulls[df_nulls['offices'].map(len) > 0]
#df_values = df_values[df_values.number_of_employees != 0]
#df_values = 
#print(len(df_values))
#display(df_values.head(10))

In [ ]:
#Función para la creación de los valores de ubicación en formato GeoJson (schools).
def column_to_list(df_school):
    lat_lon = [{"type":"Point","coordinates":[df_school['Longitude'],df_school['Latitude']]}]
    lst = lat_lon[0]
    long = lst['coordinates'][0]
    if long != None:
        return lst
    else:
        return 0